# Cleanup All Resources

Each notebook has a resource cleanup section that is designed to remove just the resources it created.

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Initial Setup` notebook.

**Resources**
- Vertex AI: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---
## Setup

Parameters:

In [18]:
PROJECT_ID = 'statmike-mlops'
REGION = 'us-central1'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

AI Platform Client Setup:

In [19]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

aiplatform.init(project=PROJECT_ID, location=REGION)

PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

---
## BigQuery
- Delete the dataset `digits`

Make a client connection to BigQuery

In [20]:
from google.cloud import bigquery
clients['bq'] = bigquery.Client(project=PROJECT_ID)

list of datasets in the project:

In [21]:
datasets = list(clients['bq'].list_datasets())

delete each dataset:

In [22]:
for ds in datasets:
    print(ds.dataset_id)
    clients['bq'].delete_dataset(ds.dataset_id,delete_contents=True,not_found_ok=True)

digits
prediction_bq_digits_code_20210722094720_2021_07_22T03_55_36_267Z
prediction_bq_digits_console_202172294628_2021_07_22T03_57_31_191Z


---
## Vertex AI > Datasets
- Delete datasets

In [23]:
datasets = aiplatform.TabularDataset.list()

In [24]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TabularDataset(dataset_name=ds.resource_name).delete()

projects/691911073727/locations/us-central1/datasets/1345635106630402048
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/1345635106630402048
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/6510499993520963584
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/1345635106630402048
projects/691911073727/locations/us-central1/datasets/6912084246060335104
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/6912084246060335104
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/7765878389650489344
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/6912084246060335104


---
## Vertex AI > Feature Store
- delete feature store including entity types and features

In [25]:
clients['fs'] = aiplatform_v1beta1.FeaturestoreServiceClient(client_options=client_options)

In [26]:
fs = clients['fs'].list_featurestores(parent=PARENT)

In [27]:
for f in fs:
    print(f.name)
    clients['fs'].delete_featurestore(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=f.name,force=True))

projects/691911073727/locations/us-central1/featurestores/digits_featurestore


---
## Vertex AI > Endpoints
- Delete endpoints forcing deployed models to undeploy

In [28]:
endpoints = aiplatform.Endpoint.list()

In [29]:
for ep in endpoints:
    print(ep.resource_name)
    aiplatform.Endpoint(endpoint_name=ep.resource_name).delete(force=True)

projects/691911073727/locations/us-central1/endpoints/2800869532317515776
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/691911073727/locations/us-central1/endpoints/2800869532317515776
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/691911073727/locations/us-central1/operations/7690443095892033536
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/691911073727/locations/us-central1/endpoints/2800869532317515776
projects/691911073727/locations/us-central1/endpoints/218055146020536320
INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/691911073727/locations/us-central1/endpoints/218055146020536320
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/218055146020536320/operations/5546729673263677440
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/691911073727/locations/us-central1/endpoin

---
## Vertex AI > Models
- Delete uploaded models

In [30]:
models = aiplatform.Model.list()

In [31]:
for model in models:
    print(model.resource_name)
    aiplatform.Model(model_name=model.resource_name).delete()

projects/691911073727/locations/us-central1/models/871525692733390848
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/871525692733390848
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/4231678582071492608
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/871525692733390848
projects/691911073727/locations/us-central1/models/3553419270832521216
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/3553419270832521216
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/4658394646764847104
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/3553419270832521216
projects/691911073727/locations/us-central1/models/3499164969071542272


---
## Vertex AI > Batch Predictions
- delete all batch prediction jobs

In [32]:
bps = aiplatform.BatchPredictionJob.list()

In [33]:
for bp in bps:
    print(bp.resource_name)
    aiplatform.BatchPredictionJob(batch_prediction_job_name=bp.resource_name).delete()

projects/691911073727/locations/us-central1/batchPredictionJobs/3978635601665589248
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/3978635601665589248
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/596147782876659712
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource name: projects/691911073727/locations/us-central1/batchPredictionJobs/3978635601665589248
projects/691911073727/locations/us-central1/batchPredictionJobs/7374349720702943232
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/7374349720702943232
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/5207833801304047616
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource nam

---
## Vertex AI > Experiments > Experiment & Tensorboard Instances
ISSUE: remove experiments first then instances.

In [4]:
clients['tb'] = aiplatform_v1beta1.TensorboardServiceClient(client_options=client_options)

In [6]:
tbs = clients['tb'].list_tensorboards(parent=PARENT)

In [ ]:
for tb in tbs:
    print(tb.name)
    tbes = clients['tb'].list_tensorboard_experiments(parent=tb.name)
    for tbe in tbes:
        print(tbe.name)
        clients['tb'].delete_tensorboard_experiment(request=aiplatform_v1beta1.types.DeleteTensorboardExperimentRequest(name=tbe.name))
    clients['tb'].delete_tensorboard(request = aiplatform_v1beta1.types.DeleteTensorboardRequest(name=tb.name))

---
## Vertex AI > Training
- delete various jobs types
- some create with aiplatform.* and other created by aiplatform.gapic.*

In [34]:
jobsA = aiplatform.CustomTrainingJob.list()
jobsA

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/2082620158542610432]

In [35]:
jobsB = aiplatform.CustomPythonPackageTrainingJob.list()
jobsB

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/2082620158542610432]

In [36]:
jobsC = aiplatform.AutoMLTabularTrainingJob.list()
jobsC

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/4474031560676343808,
 resource name: projects/691911073727/locations/us-central1/trainingPipelines/5938827339478597632]

## Vertex AI > Training > Custom Job
- Delete custom training jobs

In [37]:
clients['job'] = aiplatform.gapic.JobServiceClient(client_options=client_options)

In [38]:
for x in clients['job'].list_custom_jobs(parent=PARENT):
    print(x.display_name)
    clients['job'].delete_custom_job(name=x.name)

05_AIP_DIGITS_20210722110609
06_AIP_DIGITS_20210722103209-custom-job


## Vertex AI > Training > Training Pipeline (Python Package)
- Delete training pipelines

In [39]:
clients['pipeline'] = aiplatform.gapic.PipelineServiceClient(client_options=client_options)

In [40]:
for x in clients['pipeline'].list_training_pipelines(parent=PARENT):
    print(x.display_name)
    clients['pipeline'].delete_training_pipeline(name=x.name)

06_AIP_DIGITS_20210722103209
bq_digits_console_202172294628
bq_digits_code_20210722094720


---
## Vertex AI > Experiments > Studies
Vizier Studies

In [ ]:
clients['viz'] = aiplatform_v1beta1.VizierServiceClient(client_options=client_options)

In [ ]:
studies = clients['viz'].list_studies(parent=PARENT)

In [ ]:
for study in studies:
    print(study.name)
    clients['viz'].delete_study(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=study.name))

---
## Storage Bucket Removal

- delete contents and storage bucket `statmike-mlops` (named after project)

In [41]:
from google.cloud import storage
clients['gcs'] = storage.Client()

In [42]:
bucket = clients['gcs'].get_bucket(PROJECT_ID)

In [43]:
bucket.delete(force=True)